In [ ]:
# import packages
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
# from utils import LRN2D
# import utils
# import face_recognition
# import dlib
# import scipy.cluster.hierarchy as sch
# from sklearn.cluster import AgglomerativeClustering
# import os
import shutil
import glob
import random
# from matplotlib import pyplot as plt
# from mtcnn.mtcnn import MTCNN
import cv2
# import pafy
# import imutils
import glob, os, shutil
import time
from keras.models import load_model
from keras.utils import CustomObjectScope
import tensorflow as tf
import cv2
import os
import re
import numpy

In [ ]:
class VideoTransformer():

    def __init__(self, fps=30):
        self.fps = fps

    def video_to_frame(self, path_video, dir_frames):

        vidcap = cv2.VideoCapture(path_video)
        success,image = vidcap.read()
        count = 0
        while success:
            cv2.imwrite(dir_frames + "/frame_" + str(count).zfill(10) + ".jpg", image)     # save frame as JPEG file      
            success,image = vidcap.read()
            #print('Read a new frame: ', success)
            count += 1


    def frame_to_video(self, dir_frames, path_video, fps):
        num_img = []
        images = []
        for img in os.listdir(dir_frames):
            if img.endswith(".png") or img.endswith(".jpg"):
                # using List comprehension + isdigit() +split() 
                # getting numbers from string  
                data = img.split("_")
                data = data[1].split(".")
                num = data[0]
                #print(num)
                num_img.append(num)
                images.append(img)

        sorted_numbers = numpy.argsort(num_img)
        images_sorted = []
        #print(images)
        for i in range(len(images)):
            images_sorted.append(images[sorted_numbers[i]])

        frame = cv2.imread(os.path.join(dir_frames, images_sorted[0]))
        height, width, layers = frame.shape
        fourrc = cv2.VideoWriter_fourcc(*'MP4V')
        video = cv2.VideoWriter(path_video, fourrc, fps, (width, height))

        for image in images_sorted:
            frame = cv2.imread(os.path.join(dir_frames, image))
            video.write(frame)

        cv2.destroyAllWindows()
        video.release()

In [ ]:
import os


# Set parameters

PATH_VID = "/home/john/dlib_files/sample1.mp4"# location video 
DIR_FRAMES = "/home/john/dlib_files/frames"# directory to store frames (will be created)
PATH_VID_AN = "/home/john/dlib_files/output/sample_output_facenet2.mp4"# name of annotated video
DIR_FRAMES_AN = "/home/john/dlib_files/labeled_frames/"# directory to store annotated frames (will be created)
VID_FPS = 30

print("running")

# Create objects
# img_annotator = ImageAnnotator(PATH_CRED)
video_transf = VideoTransformer(VID_FPS)

# create folder for frames
try:
    os.mkdir(DIR_FRAMES)
except:
    print("Folder " + DIR_FRAMES + " already exists")
    for filename in os.listdir(DIR_FRAMES):
        file_path = os.path.join(DIR_FRAMES, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

try:
    os.mkdir(DIR_FRAMES_AN)
except:
    print("Folder " + DIR_FRAMES_AN + " already exists")
    for filename in os.listdir(DIR_FRAMES_AN):
        file_path = os.path.join(DIR_FRAMES_AN, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Turn video into frames
print("Turn video into frames")
video_transf.video_to_frame(PATH_VID, DIR_FRAMES)



In [ ]:
with CustomObjectScope({'tf': tf}):
    model = load_model('/home/john/Downloads/One_Shot_Learning/model/Keras_OpenFace_facenet.h5')

In [ ]:
import tensorflow as tf
import numpy as np
import os
from numpy import genfromtxt
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D


_FLOATX = 'float32'

def variable(value, dtype=_FLOATX, name=None):
  v = tf.Variable(np.asarray(value, dtype=dtype), name=name)
  _get_session().run(v.initializer)
  return v

def shape(x):
  return x.get_shape()

def square(x):
  return tf.square(x)

def zeros(shape, dtype=_FLOATX, name=None):
  return variable(np.zeros(shape), dtype, name)

def concatenate(tensors, axis=-1):
  if axis < 0:
      axis = axis % len(tensors[0].get_shape())
  return tf.concat(axis, tensors)

def LRN2D(x):
  return tf.nn.lrn(x, alpha=1e-4, beta=0.75)

def conv2d_bn(
  x,
  layer=None,
  cv1_out=None,
  cv1_filter=(1, 1),
  cv1_strides=(1, 1),
  cv2_out=None,
  cv2_filter=(3, 3),
  cv2_strides=(1, 1),
  padding=None,
):
  num = '' if cv2_out == None else '1'
  tensor = Conv2D(cv1_out, cv1_filter, strides=cv1_strides, name=layer+'_conv'+num)(x)
  tensor = BatchNormalization(axis=3, epsilon=0.00001, name=layer+'_bn'+num)(tensor)
  tensor = Activation('relu')(tensor)
  if padding == None:
    return tensor
  tensor = ZeroPadding2D(padding=padding)(tensor)
  if cv2_out == None:
    return tensor
  tensor = Conv2D(cv2_out, cv2_filter, strides=cv2_strides, name=layer+'_conv'+'2')(tensor)
  tensor = BatchNormalization(axis=3, epsilon=0.00001, name=layer+'_bn'+'2')(tensor)
  tensor = Activation('relu')(tensor)
  return tensor

weights = [
  'conv1', 'bn1', 'conv2', 'bn2', 'conv3', 'bn3',
  'inception_3a_1x1_conv', 'inception_3a_1x1_bn',
  'inception_3a_pool_conv', 'inception_3a_pool_bn',
  'inception_3a_5x5_conv1', 'inception_3a_5x5_conv2', 'inception_3a_5x5_bn1', 'inception_3a_5x5_bn2',
  'inception_3a_3x3_conv1', 'inception_3a_3x3_conv2', 'inception_3a_3x3_bn1', 'inception_3a_3x3_bn2',
  'inception_3b_3x3_conv1', 'inception_3b_3x3_conv2', 'inception_3b_3x3_bn1', 'inception_3b_3x3_bn2',
  'inception_3b_5x5_conv1', 'inception_3b_5x5_conv2', 'inception_3b_5x5_bn1', 'inception_3b_5x5_bn2',
  'inception_3b_pool_conv', 'inception_3b_pool_bn',
  'inception_3b_1x1_conv', 'inception_3b_1x1_bn',
  'inception_3c_3x3_conv1', 'inception_3c_3x3_conv2', 'inception_3c_3x3_bn1', 'inception_3c_3x3_bn2',
  'inception_3c_5x5_conv1', 'inception_3c_5x5_conv2', 'inception_3c_5x5_bn1', 'inception_3c_5x5_bn2',
  'inception_4a_3x3_conv1', 'inception_4a_3x3_conv2', 'inception_4a_3x3_bn1', 'inception_4a_3x3_bn2',
  'inception_4a_5x5_conv1', 'inception_4a_5x5_conv2', 'inception_4a_5x5_bn1', 'inception_4a_5x5_bn2',
  'inception_4a_pool_conv', 'inception_4a_pool_bn',
  'inception_4a_1x1_conv', 'inception_4a_1x1_bn',
  'inception_4e_3x3_conv1', 'inception_4e_3x3_conv2', 'inception_4e_3x3_bn1', 'inception_4e_3x3_bn2',
  'inception_4e_5x5_conv1', 'inception_4e_5x5_conv2', 'inception_4e_5x5_bn1', 'inception_4e_5x5_bn2',
  'inception_5a_3x3_conv1', 'inception_5a_3x3_conv2', 'inception_5a_3x3_bn1', 'inception_5a_3x3_bn2',
  'inception_5a_pool_conv', 'inception_5a_pool_bn',
  'inception_5a_1x1_conv', 'inception_5a_1x1_bn',
  'inception_5b_3x3_conv1', 'inception_5b_3x3_conv2', 'inception_5b_3x3_bn1', 'inception_5b_3x3_bn2',
  'inception_5b_pool_conv', 'inception_5b_pool_bn',
  'inception_5b_1x1_conv', 'inception_5b_1x1_bn',
  'dense_layer'
]

conv_shape = {
  'conv1': [64, 3, 7, 7],
  'conv2': [64, 64, 1, 1],
  'conv3': [192, 64, 3, 3],
  'inception_3a_1x1_conv': [64, 192, 1, 1],
  'inception_3a_pool_conv': [32, 192, 1, 1],
  'inception_3a_5x5_conv1': [16, 192, 1, 1],
  'inception_3a_5x5_conv2': [32, 16, 5, 5],
  'inception_3a_3x3_conv1': [96, 192, 1, 1],
  'inception_3a_3x3_conv2': [128, 96, 3, 3],
  'inception_3b_3x3_conv1': [96, 256, 1, 1],
  'inception_3b_3x3_conv2': [128, 96, 3, 3],
  'inception_3b_5x5_conv1': [32, 256, 1, 1],
  'inception_3b_5x5_conv2': [64, 32, 5, 5],
  'inception_3b_pool_conv': [64, 256, 1, 1],
  'inception_3b_1x1_conv': [64, 256, 1, 1],
  'inception_3c_3x3_conv1': [128, 320, 1, 1],
  'inception_3c_3x3_conv2': [256, 128, 3, 3],
  'inception_3c_5x5_conv1': [32, 320, 1, 1],
  'inception_3c_5x5_conv2': [64, 32, 5, 5],
  'inception_4a_3x3_conv1': [96, 640, 1, 1],
  'inception_4a_3x3_conv2': [192, 96, 3, 3],
  'inception_4a_5x5_conv1': [32, 640, 1, 1,],
  'inception_4a_5x5_conv2': [64, 32, 5, 5],
  'inception_4a_pool_conv': [128, 640, 1, 1],
  'inception_4a_1x1_conv': [256, 640, 1, 1],
  'inception_4e_3x3_conv1': [160, 640, 1, 1],
  'inception_4e_3x3_conv2': [256, 160, 3, 3],
  'inception_4e_5x5_conv1': [64, 640, 1, 1],
  'inception_4e_5x5_conv2': [128, 64, 5, 5],
  'inception_5a_3x3_conv1': [96, 1024, 1, 1],
  'inception_5a_3x3_conv2': [384, 96, 3, 3],
  'inception_5a_pool_conv': [96, 1024, 1, 1],
  'inception_5a_1x1_conv': [256, 1024, 1, 1],
  'inception_5b_3x3_conv1': [96, 736, 1, 1],
  'inception_5b_3x3_conv2': [384, 96, 3, 3],
  'inception_5b_pool_conv': [96, 736, 1, 1],
  'inception_5b_1x1_conv': [256, 736, 1, 1],
}

def load_weights():
  # Set weights path
  dirPath = '/home/john/dlib_files/facenet/weights'
  fileNames = filter(lambda f: not f.startswith('.'), os.listdir(dirPath))
  paths = {}
  weights_dict = {}

  for n in fileNames:
    paths[n.replace('.csv', '')] = dirPath + '/' + n

  for name in weights:
    if 'conv' in name:
      conv_w = genfromtxt(paths[name + '_w'], delimiter=',', dtype=None)
      conv_w = np.reshape(conv_w, conv_shape[name])
      conv_w = np.transpose(conv_w, (2, 3, 1, 0))
      conv_b = genfromtxt(paths[name + '_b'], delimiter=',', dtype=None)
      weights_dict[name] = [conv_w, conv_b]     
    elif 'bn' in name:
      bn_w = genfromtxt(paths[name + '_w'], delimiter=',', dtype=None)
      bn_b = genfromtxt(paths[name + '_b'], delimiter=',', dtype=None)
      bn_m = genfromtxt(paths[name + '_m'], delimiter=',', dtype=None)
      bn_v = genfromtxt(paths[name + '_v'], delimiter=',', dtype=None)
      weights_dict[name] = [bn_w, bn_b, bn_m, bn_v]
    elif 'dense' in name:
      dense_w = genfromtxt(dirPath+'/dense_w.csv', delimiter=',', dtype=None)
      dense_w = np.reshape(dense_w, (128, 736))
      dense_w = np.transpose(dense_w, (1, 0))
      dense_b = genfromtxt(dirPath+'/dense_b.csv', delimiter=',', dtype=None)
      weights_dict[name] = [dense_w, dense_b]

  return weights_dict



In [ ]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

In [ ]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

#         print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance <= 0.8:
        return str(name)
    else:
        return 

In [ ]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("/home/john/dlib_files/unique_faces/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

In [ ]:
def recognize_faces_in_images1(input_embeddings):
    
    face_cascade = cv2.CascadeClassifier('/home/john/dlib_files/haarcascade_frontalface_default.xml')
    unknown_counter = 0
    font = cv2.FONT_HERSHEY_SIMPLEX
    for image in os.listdir("/home/john/dlib_files/frames_R"):
        image_path = os.path.join("/home/john/dlib_files/frames_R",image)
        image_name = image_path.split('/')[-1].strip('.jpg')
        frame = cv2.imread(image_path)
#         img = frame
        height, width, channels = frame.shape
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(0,255,0),2)
                cv2.putText(frame, str(identity), (x1+5,y1-5), font, 1, (0,255,0), 2,cv2.LINE_AA)
                name = "/home/john/dlib_files/labeled_frames/"+ str(image_name) + '.jpg'
                cv2.imwrite(name, frame)
            else:
                cv2.rectangle(frame,(x1, y1),(x2, y2), (0, 0, 255), 2)
                name = "/home/john/dlib_files/labeled_frames/"+ str(image_name) + '.jpg'
                cv2.imwrite(name, frame)
        name = "/home/john/dlib_files/labeled_frames/"+ str(image_name) + '.jpg'
        cv2.imwrite(name, frame)

        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", frame)

        if key == 27: # exit on ESC
            break
#     vc.release()
    cv2.destroyAllWindows()

In [ ]:
import cv2
import os
import re
import numpy

class VideoTransformer():

    def __init__(self, fps=30):
        self.fps = fps

    def video_to_frame(self, path_video, dir_frames):

        vidcap = cv2.VideoCapture(path_video)
        success,image = vidcap.read()
        count = 0
        while success:
            cv2.imwrite(dir_frames + "/frame_" + str(count).zfill(10) + ".jpg", image)     # save frame as JPEG file      
            success,image = vidcap.read()
            #print('Read a new frame: ', success)
            count += 1


    def frame_to_video(self, dir_frames, path_video, fps):
        num_img = []
        images = []
        for img in os.listdir(dir_frames):
            if img.endswith(".png") or img.endswith(".jpg"):
                # using List comprehension + isdigit() +split() 
                # getting numbers from string  
                data = img.split("_")
                data = data[1].split(".")
                num = data[0]
                #print(num)
                num_img.append(num)
                images.append(img)

        sorted_numbers = numpy.argsort(num_img)
        images_sorted = []
        #print(images)
        for i in range(len(images)):
            images_sorted.append(images[sorted_numbers[i]])

        frame = cv2.imread(os.path.join(dir_frames, images_sorted[0]))
        height, width, layers = frame.shape
        fourrc = cv2.VideoWriter_fourcc(*'MP4V')
        video = cv2.VideoWriter(path_video, fourrc, fps, (width, height))

        for image in images_sorted:
            frame = cv2.imread(os.path.join(dir_frames, image))
            video.write(frame)

        cv2.destroyAllWindows()
        video.release()

import os


# Set parameters

PATH_VID = "/home/john/dlib_files/sample1.mp4"# location video 
DIR_FRAMES = "/home/john/dlib_files/frames_R"# directory to store frames (will be created)
PATH_VID_AN = "/home/john/dlib_files/output/sample_output_facenet2.mp4"# name of annotated video
DIR_FRAMES_AN = "/home/john/dlib_files/labeled_frames/"# directory to store annotated frames (will be created)
VID_FPS = 30

print("running")

# Create objects
# img_annotator = ImageAnnotator(PATH_CRED)
video_transf = VideoTransformer(VID_FPS)

# create folder for frames
try:
    os.mkdir(DIR_FRAMES)
except:
    print("Folder " + DIR_FRAMES + " already exists")
    for filename in os.listdir(DIR_FRAMES):
        file_path = os.path.join(DIR_FRAMES, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

try:
    os.mkdir(DIR_FRAMES_AN)
except:
    print("Folder " + DIR_FRAMES_AN + " already exists")
    for filename in os.listdir(DIR_FRAMES_AN):
        file_path = os.path.join(DIR_FRAMES_AN, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Turn video into frames
print("Turn video into frames")
video_transf.video_to_frame(PATH_VID, DIR_FRAMES)



# import packages
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
# from utils import LRN2D
# import utils
# import face_recognition
# import dlib
# import scipy.cluster.hierarchy as sch
# from sklearn.cluster import AgglomerativeClustering
# import os
import shutil
import glob
import random
# from matplotlib import pyplot as plt
# from mtcnn.mtcnn import MTCNN
import cv2
# import pafy
# import imutils
import glob, os, shutil
import time
from keras.models import load_model
from keras.utils import CustomObjectScope
import tensorflow as tf

with CustomObjectScope({'tf': tf}):
    model = load_model('/home/john/Downloads/One_Shot_Learning/model/Keras_OpenFace_facenet.h5')

import tensorflow as tf
import numpy as np
import os
from numpy import genfromtxt
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D


_FLOATX = 'float32'

def variable(value, dtype=_FLOATX, name=None):
  v = tf.Variable(np.asarray(value, dtype=dtype), name=name)
  _get_session().run(v.initializer)
  return v

def shape(x):
  return x.get_shape()

def square(x):
  return tf.square(x)

def zeros(shape, dtype=_FLOATX, name=None):
  return variable(np.zeros(shape), dtype, name)

def concatenate(tensors, axis=-1):
  if axis < 0:
      axis = axis % len(tensors[0].get_shape())
  return tf.concat(axis, tensors)

def LRN2D(x):
  return tf.nn.lrn(x, alpha=1e-4, beta=0.75)

def conv2d_bn(
  x,
  layer=None,
  cv1_out=None,
  cv1_filter=(1, 1),
  cv1_strides=(1, 1),
  cv2_out=None,
  cv2_filter=(3, 3),
  cv2_strides=(1, 1),
  padding=None,
):
  num = '' if cv2_out == None else '1'
  tensor = Conv2D(cv1_out, cv1_filter, strides=cv1_strides, name=layer+'_conv'+num)(x)
  tensor = BatchNormalization(axis=3, epsilon=0.00001, name=layer+'_bn'+num)(tensor)
  tensor = Activation('relu')(tensor)
  if padding == None:
    return tensor
  tensor = ZeroPadding2D(padding=padding)(tensor)
  if cv2_out == None:
    return tensor
  tensor = Conv2D(cv2_out, cv2_filter, strides=cv2_strides, name=layer+'_conv'+'2')(tensor)
  tensor = BatchNormalization(axis=3, epsilon=0.00001, name=layer+'_bn'+'2')(tensor)
  tensor = Activation('relu')(tensor)
  return tensor

weights = [
  'conv1', 'bn1', 'conv2', 'bn2', 'conv3', 'bn3',
  'inception_3a_1x1_conv', 'inception_3a_1x1_bn',
  'inception_3a_pool_conv', 'inception_3a_pool_bn',
  'inception_3a_5x5_conv1', 'inception_3a_5x5_conv2', 'inception_3a_5x5_bn1', 'inception_3a_5x5_bn2',
  'inception_3a_3x3_conv1', 'inception_3a_3x3_conv2', 'inception_3a_3x3_bn1', 'inception_3a_3x3_bn2',
  'inception_3b_3x3_conv1', 'inception_3b_3x3_conv2', 'inception_3b_3x3_bn1', 'inception_3b_3x3_bn2',
  'inception_3b_5x5_conv1', 'inception_3b_5x5_conv2', 'inception_3b_5x5_bn1', 'inception_3b_5x5_bn2',
  'inception_3b_pool_conv', 'inception_3b_pool_bn',
  'inception_3b_1x1_conv', 'inception_3b_1x1_bn',
  'inception_3c_3x3_conv1', 'inception_3c_3x3_conv2', 'inception_3c_3x3_bn1', 'inception_3c_3x3_bn2',
  'inception_3c_5x5_conv1', 'inception_3c_5x5_conv2', 'inception_3c_5x5_bn1', 'inception_3c_5x5_bn2',
  'inception_4a_3x3_conv1', 'inception_4a_3x3_conv2', 'inception_4a_3x3_bn1', 'inception_4a_3x3_bn2',
  'inception_4a_5x5_conv1', 'inception_4a_5x5_conv2', 'inception_4a_5x5_bn1', 'inception_4a_5x5_bn2',
  'inception_4a_pool_conv', 'inception_4a_pool_bn',
  'inception_4a_1x1_conv', 'inception_4a_1x1_bn',
  'inception_4e_3x3_conv1', 'inception_4e_3x3_conv2', 'inception_4e_3x3_bn1', 'inception_4e_3x3_bn2',
  'inception_4e_5x5_conv1', 'inception_4e_5x5_conv2', 'inception_4e_5x5_bn1', 'inception_4e_5x5_bn2',
  'inception_5a_3x3_conv1', 'inception_5a_3x3_conv2', 'inception_5a_3x3_bn1', 'inception_5a_3x3_bn2',
  'inception_5a_pool_conv', 'inception_5a_pool_bn',
  'inception_5a_1x1_conv', 'inception_5a_1x1_bn',
  'inception_5b_3x3_conv1', 'inception_5b_3x3_conv2', 'inception_5b_3x3_bn1', 'inception_5b_3x3_bn2',
  'inception_5b_pool_conv', 'inception_5b_pool_bn',
  'inception_5b_1x1_conv', 'inception_5b_1x1_bn',
  'dense_layer'
]

conv_shape = {
  'conv1': [64, 3, 7, 7],
  'conv2': [64, 64, 1, 1],
  'conv3': [192, 64, 3, 3],
  'inception_3a_1x1_conv': [64, 192, 1, 1],
  'inception_3a_pool_conv': [32, 192, 1, 1],
  'inception_3a_5x5_conv1': [16, 192, 1, 1],
  'inception_3a_5x5_conv2': [32, 16, 5, 5],
  'inception_3a_3x3_conv1': [96, 192, 1, 1],
  'inception_3a_3x3_conv2': [128, 96, 3, 3],
  'inception_3b_3x3_conv1': [96, 256, 1, 1],
  'inception_3b_3x3_conv2': [128, 96, 3, 3],
  'inception_3b_5x5_conv1': [32, 256, 1, 1],
  'inception_3b_5x5_conv2': [64, 32, 5, 5],
  'inception_3b_pool_conv': [64, 256, 1, 1],
  'inception_3b_1x1_conv': [64, 256, 1, 1],
  'inception_3c_3x3_conv1': [128, 320, 1, 1],
  'inception_3c_3x3_conv2': [256, 128, 3, 3],
  'inception_3c_5x5_conv1': [32, 320, 1, 1],
  'inception_3c_5x5_conv2': [64, 32, 5, 5],
  'inception_4a_3x3_conv1': [96, 640, 1, 1],
  'inception_4a_3x3_conv2': [192, 96, 3, 3],
  'inception_4a_5x5_conv1': [32, 640, 1, 1,],
  'inception_4a_5x5_conv2': [64, 32, 5, 5],
  'inception_4a_pool_conv': [128, 640, 1, 1],
  'inception_4a_1x1_conv': [256, 640, 1, 1],
  'inception_4e_3x3_conv1': [160, 640, 1, 1],
  'inception_4e_3x3_conv2': [256, 160, 3, 3],
  'inception_4e_5x5_conv1': [64, 640, 1, 1],
  'inception_4e_5x5_conv2': [128, 64, 5, 5],
  'inception_5a_3x3_conv1': [96, 1024, 1, 1],
  'inception_5a_3x3_conv2': [384, 96, 3, 3],
  'inception_5a_pool_conv': [96, 1024, 1, 1],
  'inception_5a_1x1_conv': [256, 1024, 1, 1],
  'inception_5b_3x3_conv1': [96, 736, 1, 1],
  'inception_5b_3x3_conv2': [384, 96, 3, 3],
  'inception_5b_pool_conv': [96, 736, 1, 1],
  'inception_5b_1x1_conv': [256, 736, 1, 1],
}

def load_weights():
  # Set weights path
  dirPath = '/home/john/dlib_files/facenet/weights'
  fileNames = filter(lambda f: not f.startswith('.'), os.listdir(dirPath))
  paths = {}
  weights_dict = {}

  for n in fileNames:
    paths[n.replace('.csv', '')] = dirPath + '/' + n

  for name in weights:
    if 'conv' in name:
      conv_w = genfromtxt(paths[name + '_w'], delimiter=',', dtype=None)
      conv_w = np.reshape(conv_w, conv_shape[name])
      conv_w = np.transpose(conv_w, (2, 3, 1, 0))
      conv_b = genfromtxt(paths[name + '_b'], delimiter=',', dtype=None)
      weights_dict[name] = [conv_w, conv_b]     
    elif 'bn' in name:
      bn_w = genfromtxt(paths[name + '_w'], delimiter=',', dtype=None)
      bn_b = genfromtxt(paths[name + '_b'], delimiter=',', dtype=None)
      bn_m = genfromtxt(paths[name + '_m'], delimiter=',', dtype=None)
      bn_v = genfromtxt(paths[name + '_v'], delimiter=',', dtype=None)
      weights_dict[name] = [bn_w, bn_b, bn_m, bn_v]
    elif 'dense' in name:
      dense_w = genfromtxt(dirPath+'/dense_w.csv', delimiter=',', dtype=None)
      dense_w = np.reshape(dense_w, (128, 736))
      dense_w = np.transpose(dense_w, (1, 0))
      dense_b = genfromtxt(dirPath+'/dense_b.csv', delimiter=',', dtype=None)
      weights_dict[name] = [dense_w, dense_b]

  return weights_dict



def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

#         print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance <= 0.8:
        return str(name)
    else:
        return 

import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("/home/john/dlib_files/unique_faces/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_images1(input_embeddings):
    
    face_cascade = cv2.CascadeClassifier('/home/john/dlib_files/haarcascade_frontalface_default.xml')
    unknown_counter = 0
    font = cv2.FONT_HERSHEY_SIMPLEX
    for image in os.listdir("/home/john/dlib_files/frames"):
        image_path = os.path.join("/home/john/dlib_files/frames",image)
        image_name = image_path.split('/')[-1].strip('.jpg')
        frame = cv2.imread(image_path)
#         img = frame
        height, width, channels = frame.shape
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(0,255,0),2)
                cv2.putText(frame, str(identity), (x1+5,y1-5), font, 1, (0,255,0), 2,cv2.LINE_AA)
                name = "/home/john/dlib_files/labeled_frames/"+ str(image_name) + '.jpg'
                cv2.imwrite(name, frame)
            else:
                cv2.rectangle(frame,(x1, y1),(x2, y2), (0, 0, 255), 2)
                name = "/home/john/dlib_files/labeled_frames/"+ str(image_name) + '.jpg'
                cv2.imwrite(name, frame)
        name = "/home/john/dlib_files/labeled_frames/"+ str(image_name) + '.jpg'
        cv2.imwrite(name, frame)

        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", frame)

        if key == 27: # exit on ESC
            break
#     vc.release()
    cv2.destroyAllWindows()

In [ ]:
t1 = time.perf_counter()

input_embeddings = create_input_image_embeddings()
recognize_faces_in_images1(input_embeddings)

t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

In [ ]:
# transform frames to video
print("Transform frames to video")
video_transf.frame_to_video(DIR_FRAMES_AN, PATH_VID_AN,VID_FPS)

print("Done!")